In [ ]:
from MZPackage.AdversarialGenerator import AdversarialGenerator
from MZPackage.Utilities import *

In [ ]:
ag = AdversarialGenerator()

### PCA

In [ ]:
modelType = "pca"
count = 500
attackSamples = ag.generateAttackSamples(modelType, count)
saveData(attackSamples, "adversarial_data_set_" + modelType)

### Fully Connected

In [ ]:
modelType = "fullyConnected"
count = 500
attackSamples = ag.generateAttackSamples(modelType, count)
saveData(attackSamples, "adversarial_data_set_" + modelType)

### 1D Conv

In [ ]:
modelType = "conv"
count = 500
attackSamples = ag.generateAttackSamplesLstm(modelType, count)
saveData(attackSamples, "adversarial_data_set_" + modelType)

### LSTM

In [ ]:
## NOT WORKING FOR LSTM : USE THE CELL BELOW
modelType = "lstm"
count = 500
attackSamples = ag.generateAttackSamples(modelType, count)
saveData(attackSamples, "adversarial_data_set_" + modelType)

### For LSTM, the above code won't work but this works

In [ ]:
from tensorflow.keras.models import load_model
# Convert sequential to functional model
from tensorflow.keras import layers, models
import tensorflow as tf
import tqdm

config = loadConfig()        
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]
thresholds = loadData("thresholds")

model = load_model('Trained_Model/lstm.h5')



input_layer = layers.Input(batch_shape=model.layers[0].input_shape)
prev_layer = input_layer
for layer in model.layers:
    prev_layer = layer(prev_layer)

funcmodel = models.Model([input_layer], [prev_layer])

@tf.function # To fix: 'Tensor' object has no attribute 'numpy' 
def create_adversarial_pattern_lstm(input_image, input_label):
    loss_object = tf.keras.losses.MeanSquaredError()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = funcmodel(input_image)
        #loss = loss_object(input_label, prediction)
        loss = loss_object(input_image, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

@tf.function 
def mzAttackSampleGeneratorFromZero(iterations,seed,epsilon):
    tfInput = tfoutput = seed
    for step in range(0,iterations):
        tfInput = seed
        tfOutput = tfInput
        sgrad = create_adversarial_pattern_lstm(tfInput, tfOutput)                        
        perturbations = epsilon*sgrad
        seed = seed - perturbations    
        #print(perturbations.shape)
        #print(adv_x.shape)    
        #plt.plot(adv_x[0,:,6])
        #plt.figure()    
    attackSample = seed
    return attackSample



attackSamples = list()
# Generating attack samples from random seeds
for i in tqdm.tqdm(range(500)):
    randSample = np.random.rand(1,sequenceLen,dimensionsCount)
    mse = 1000
    attackSample = randSample
    while mse > thresholds["lstm"]:
        attackSample = mzAttackSampleGeneratorFromZero(1,attackSample,0.001)
        predicted = model.predict(attackSample)
        mse = (np.square(attackSample - predicted)).mean(axis=2).mean(axis=1)        
    attackSamples.append(attackSample)

In [ ]:
dataArray = np.array(attackSamples)
saveData(dataArray, "adversarial_data_set_lstm")